In [5]:
import pandas as pd

# Charger le fichier
file_path = "fanorona_dataset.csv"
df = pd.read_csv(file_path)

# Afficher les premières lignes pour examiner la structure
df.head()

,Type,Board
0,1,"[[-1, -1, -1], [0, 0, 0], [0, 0, 0]]"
1,1,"[[0, -1, 0], [0, -1, 0], [0, -1, 0]]"
2,1,"[[0, 0, -1], [0, -1, 0], [-1, 0, 0]]"
3,1,"[[0, 0, -1], [0, -1, 0], [-1, 0, 0]]"
4,1,"[[0, 0, 0], [1, 1, 1], [0, 0, 0]]"


In [6]:
# Vérifier la répartition des types
type_counts = df["Type"].value_counts()

# Vérifier le nombre de doublons exacts
duplicate_rows = df.duplicated().sum()

# Vérifier les doublons uniquement sur la colonne "Board" (états de jeu)
duplicate_boards = df.duplicated(subset=["Board"]).sum()

type_counts, duplicate_rows, duplicate_boards

(Type
 1    500
 0    500
 Name: count, dtype: int64,
 np.int64(968),
 np.int64(968))

In [7]:
# Supprimer les doublons tout en gardant 500 gagnants et 500 perdants
df_unique = df.drop_duplicates(subset=["Board"])

# Vérifier combien de lignes uniques restent
df_unique_count = len(df_unique)

# Si on a moins de 1000 lignes après suppression des doublons, compléter avec des copies légères modifiées
df_unique_count

32

In [8]:
import ast
import numpy as np

# Fonction pour effectuer une rotation de 90° sur un plateau
def rotate_board(board):
    return np.rot90(board).tolist()

# Fonction pour inverser un plateau horizontalement
def flip_board(board):
    return np.fliplr(board).tolist()

# Convertir les plateaux de jeu de chaîne en liste pour manipulation
df_unique["Board"] = df_unique["Board"].apply(ast.literal_eval)

# Générer des variations des plateaux pour augmenter la diversité
augmented_boards = []
for _, row in df_unique.iterrows():
    board = np.array(row["Board"])
    augmented_boards.append((row["Type"], board.tolist()))  # Original
    augmented_boards.append((row["Type"], rotate_board(board)))  # Rotation 90°
    augmented_boards.append((row["Type"], flip_board(board)))  # Flip horizontal
    augmented_boards.append((row["Type"], rotate_board(rotate_board(board))))  # Rotation 180°

# Convertir en DataFrame et limiter à 1000 lignes
df_augmented = pd.DataFrame(augmented_boards, columns=["Type", "Board"]).head(1000)

# Vérifier la nouvelle répartition des données
df_augmented["Type"].value_counts()

C:\Users\LIANTSOA\AppData\Local\Temp\ipykernel_11712\1194714045.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique["Board"] = df_unique["Board"].apply(ast.literal_eval)


Type
1    64
0    64
Name: count, dtype: int64

In [9]:
# Appliquer davantage de transformations pour enrichir les données
more_augmented_boards = []

for _, row in df_unique.iterrows():
    board = np.array(row["Board"])
    
    # Générer plusieurs variations
    transformations = [
        board.tolist(),
        rotate_board(board),
        flip_board(board),
        rotate_board(rotate_board(board)),
        rotate_board(flip_board(board)),
        flip_board(rotate_board(board))
    ]
    
    for transformed_board in transformations:
        more_augmented_boards.append((row["Type"], transformed_board))

# Convertir en DataFrame et limiter à 1000 lignes équilibrées
df_more_augmented = pd.DataFrame(more_augmented_boards, columns=["Type", "Board"])

# Assurer un équilibre entre gagnants et perdants
df_balanced = pd.concat([
    df_more_augmented[df_more_augmented["Type"] == 1].head(500),
    df_more_augmented[df_more_augmented["Type"] == 0].head(500)
])

# Vérifier la répartition finale
df_balanced["Type"].value_counts()


Type
1    96
0    96
Name: count, dtype: int64

In [14]:
# Générer plus de transformations en combinant rotation et flip de manière plus exhaustive
final_augmented_boards = []

for _, row in df_unique.iterrows():
    board = np.array(row["Board"])
    
    # Appliquer plus de variations sur chaque plateau
    transformations = [
        board.tolist(),
        rotate_board(board),
        rotate_board(rotate_board(board)),
        rotate_board(rotate_board(rotate_board(board))),
        flip_board(board),
        rotate_board(flip_board(board)),
        rotate_board(rotate_board(flip_board(board))),
        rotate_board(rotate_board(rotate_board(flip_board(board)))),
        flip_board(rotate_board(board)),
        flip_board(rotate_board(rotate_board(board))),
        flip_board(rotate_board(rotate_board(rotate_board(board))))
    ]
    
    for transformed_board in transformations:
        final_augmented_boards.append((row["Type"], transformed_board))

# Convertir en DataFrame et limiter à 1000 lignes équilibrées
df_final = pd.DataFrame(final_augmented_boards, columns=["Type", "Board"])

# Assurer un équilibre entre gagnants et perdants (500 chacun)
df_final_balanced = pd.concat([
    df_final[df_final["Type"] == 1].head(500),
    df_final[df_final["Type"] == 0].head(500)
])

# Vérifier la répartition finale
df_final_balanced["Type"].value_counts()


Type
1    176
0    176
Name: count, dtype: int64

In [15]:
import random

# Fonction pour introduire une petite perturbation dans le plateau de jeu
def perturb_board(board):
    perturbed_board = np.array(board).copy()
    x, y = random.randint(0, 2), random.randint(0, 2)  # Choisir une position aléatoire
    perturbed_board[x, y] = random.choice([-1, 0, 1])  # Modifier légèrement la valeur
    return perturbed_board.tolist()

# Générer plus de variations avec perturbations aléatoires
extra_augmented_boards = []
for _, row in df_final.iterrows():
    board = row["Board"]
    extra_augmented_boards.append((row["Type"], board))  # Original
    extra_augmented_boards.append((row["Type"], perturb_board(board)))  # Légèrement modifié

# Convertir en DataFrame et équilibrer à 1000 lignes (500 gagnants, 500 perdants)
df_extra = pd.DataFrame(extra_augmented_boards, columns=["Type", "Board"])

df_final_balanced = pd.concat([
    df_extra[df_extra["Type"] == 1].head(500),
    df_extra[df_extra["Type"] == 0].head(500)
])

# Vérifier la répartition finale
df_final_balanced["Type"].value_counts()


Type
1    352
0    352
Name: count, dtype: int64

In [16]:
# Générer encore plus de variations avec des perturbations supplémentaires
while len(df_final_balanced) < 1000:
    for _, row in df_extra.iterrows():
        if len(df_final_balanced) >= 1000:
            break
        perturbed_board = perturb_board(row["Board"])
        df_final_balanced = pd.concat([df_final_balanced, pd.DataFrame([(row["Type"], perturbed_board)], columns=["Type", "Board"])])

# Vérifier la répartition finale
df_final_balanced["Type"].value_counts()


Type
1    648
0    352
Name: count, dtype: int64

In [17]:
# Générer plus de variations pour les perdants uniquement
extra_losing_boards = df_extra[df_extra["Type"] == 0].sample(n=148, replace=True, random_state=42)

# Ajouter ces nouvelles données aux perdants pour équilibrer
df_final_balanced = pd.concat([df_final_balanced, extra_losing_boards]).head(1000)

# Vérifier la répartition finale
df_final_balanced["Type"].value_counts()


Type
1    648
0    352
Name: count, dtype: int64

In [18]:
# Générer des variations supplémentaires pour les perdants jusqu'à obtenir 500
while df_final_balanced["Type"].value_counts()[0] < 500:
    for _, row in df_extra[df_extra["Type"] == 0].iterrows():
        if df_final_balanced["Type"].value_counts()[0] >= 500:
            break
        perturbed_board = perturb_board(row["Board"])
        df_final_balanced = pd.concat([df_final_balanced, pd.DataFrame([(row["Type"], perturbed_board)], columns=["Type", "Board"])])

# Vérifier la répartition finale après équilibrage
df_final_balanced["Type"].value_counts()


Type
1    648
0    500
Name: count, dtype: int64

In [19]:
# Réduire le nombre de gagnants à 500
df_final_balanced = pd.concat([
    df_final_balanced[df_final_balanced["Type"] == 1].head(500),
    df_final_balanced[df_final_balanced["Type"] == 0].head(500)
])

# Vérifier la répartition finale après équilibrage
df_final_balanced["Type"].value_counts()


Type
1    500
0    500
Name: count, dtype: int64

In [20]:
# Sauvegarder le dataset final
final_file_path = "fanorona_dataset_corrected.csv"
df_final_balanced.to_csv(final_file_path, index=False)

# Retourner le chemin du fichier
final_file_path


'fanorona_dataset_corrected.csv'

In [21]:
# Recharger le fichier corrigé pour analyser la structure
df_corrected = pd.read_csv(final_file_path)

# Afficher les premières lignes pour vérifier l'ordre des colonnes
df_corrected.head()


,Type,Board
0,1,"[[-1, -1, -1], [0, 0, 0], [0, 0, 0]]"
1,1,"[[-1, -1, -1], [0, 0, 0], [0, 0, -1]]"
2,1,"[[-1, 0, 0], [-1, 0, 0], [-1, 0, 0]]"
3,1,"[[-1, 0, 0], [-1, 0, 0], [-1, 0, 0]]"
4,1,"[[0, 0, 0], [0, 0, 0], [-1, -1, -1]]"


In [22]:
# Reformater le fichier CSV pour éviter les erreurs d'interprétation
# S'assurer que "Type" est bien la première colonne et est stocké en int
df_corrected["Type"] = df_corrected["Type"].astype(int)

# Sauvegarder une nouvelle version propre du fichier
cleaned_file_path = "fanorona_dataset_cleaned.csv"
df_corrected.to_csv(cleaned_file_path, index=False, header=False)  # Supprimer l'en-tête pour éviter d'éventuels problèmes

# Retourner le chemin du fichier
cleaned_file_path
print("fichier nettoyé")

fichier nettoyé
